In [ ]:
%run -i code/utils.py

# 1. Reading Elevation File

In [ ]:
name_of_file = 'Data/chimmey_tops_dem_1m_utm17n.csv'
Mst_original = read_elevation(name_of_file)

# 2. Creating the input FDS file
## 2.1 Defining the Region Where the Simulation Takes Place

In [ ]:
Min_x = 400
Max_x = 600
Min_y = 0
Max_y = 200
Mst   = Mst_original.copy()
Mst = Mst[(Mst.x > Min_x-1) & (Mst.x < Max_x) & (Mst.y > Min_y-1) & (Mst.y < Max_y)]

# 2.2 Defining the Resolution of the Meshes and the obstacles in Meters

In [ ]:
# Comments
R = 4                                   # Define the resolution of the cells
Mst = Mst[(Mst.x%R==0) & (Mst.y%R==0)]  # Filtering the data 
Ro = 4                                  # Define the resolution of the obstacles


## 2.3 Calculating the Range of the Elevation 

In [ ]:
# Minimum and Maximum elevation
Min_z = math.floor(Mst['Elevation'].min())-4
Max_z = math.ceil(Mst['Elevation'].max())


# 2.4 Writing the FDS File 

In [ ]:
# Defining the time variables
T_begin    = 0.0
DTT        = 60.0
T_end      = T_begin + DTT
DT         = 0.1

# Number of meshes in x, y, z directions
Nmx        = 2     
Nmy        = 2     
Nmz        = 2 

PC         = 1         # Predictor-Corrector Status Flag
Location   = [500,100] # Initial Location of the fire
Child      = 'Region1' # Start of Sequential Domain Decomposition
HRRPUA     = 1500      # Heat Release Rate (HRR)

# File name conventions
foldername = f'chimney'
filename   = f'{Child}.fds'

# Defining the dataframe with the HRR information
Hrr = Mst[(Mst['x'] < Location[0]+5) & (Mst['x'] > Location[0]-5) & (Mst['y'] < Location[1]+5) & (Mst['y'] > Location[1]-5)]
Hrr = Hrr[[Hrr.columns[0],Hrr.columns[1]]]
Hrr['hrr'] = HRRPUA

# Function to write input FDS file with parameter specified above
write_fds_file(T_begin, T_end, DT, PC, Nmx, Nmy, Nmz, Hrr, Child)

# 3 Running the FDS program

## 3.1 Defining the folder where the FDS input file is located

In [ ]:
PATH =  os.getcwd()
Bin_folder = f"{PATH}/Bin"
FDS_FOLDER = f"FDSFiles/{foldername}"
os.chdir(FDS_FOLDER)

## 3.2 Running the FDS program

In [ ]:
number_of_process = 4                 # Number of processes(cores) needed to run the created meshes
os.environ['OMP_NUM_THREADS'] = '4'   # Sets OpenMP Threads to 4

In [ ]:
os.chdir(f'{PATH}/{FDS_FOLDER}')
os.system(f"mpiexec -n {number_of_process} fds {filename}")

# 4 Converting the HRR Information from the plot3d Files to CSV Files

In [ ]:
os.chdir(PATH)
Number_of_meshes = Nmx*Nmy*Nmz

reading_hrr(Child, Number_of_meshes)

# 5 Extracting the HRR from the CSV Files

## 5.1 Converting the CSV Files into Dataframes

In [ ]:
os.chdir(PATH)
Hrr = pd.DataFrame()
for i in range(1,Number_of_meshes+1):
    temp = pd.read_csv(f"{FDS_FOLDER}/{Child}_{i}.csv")
    temp = temp.drop(temp.index[0])
    temp = temp.reset_index(drop=True)
    Hrr  = Hrr.append(temp)
    Hrr = Hrr.reset_index(drop=True)
Hrr = Hrr[[Hrr.columns[0],Hrr.columns[1],Hrr.columns[2],Hrr.columns[7]]]
Hrr.columns = ['x','y','z','hrr']

## 5.2 Cleaning the Data and Matching the Format of Numbers

In [ ]:
Hrr["hrr"] = pd.to_numeric(Hrr["hrr"], downcast="float")
Hrr["x"] = pd.to_numeric(Hrr["x"], downcast="float")
Hrr["y"] = pd.to_numeric(Hrr["y"], downcast="float")
Hrr["z"] = pd.to_numeric(Hrr["z"], downcast="float")
Mst["x"] = pd.to_numeric(Mst["x"], downcast="float")
Mst["y"] = pd.to_numeric(Mst["y"], downcast="float")
Mst.shape

## 5.3 Extracting the values of HRR greater than 10

In [ ]:
Hrr = Hrr[Hrr['hrr']>10]

# 6 Defining the cells with fire

In [ ]:
Min_x = 300
Max_x = 500
Min_y = 100
Max_y = 300
Mst   = Mst_original.copy()
Mst = Mst[(Mst.x > Min_x-1) & (Mst.x < Max_x) & (Mst.y > Min_y-1) & (Mst.y < Max_y)]
Mst = Mst[(Mst.x%R==0) & (Mst.y%R==0)]
# Minimum and Maximum elevation
Min_z = math.floor(Mst['Elevation'].min())-4
Max_z = math.ceil(Mst['Elevation'].max())

In [ ]:
T_begin = T_end
T_end   = T_begin+DTT
Child   = 'Region2'
foldername = f'chimney'
filename   = f'{Child}.fds'

In [ ]:
restart_fds_file(T_begin, T_end, DT, PC, Nmx, Nmy, Nmz, Hrr, Child)

In [ ]:
os.chdir(PATH)
os.chdir(FDS_FOLDER)
number_of_process = 4
os.system(f"mpiexec -n {number_of_process} fds {filename}")
# bash(['mpiexec', '-n', number_of_process,'fds',filename])